### Use MLFlow for experiment tracking & logging the models
- Use MLflow client to log parameters, metrics and trained model
- Use Azure ML as the MLflow server - params/metrics and model will be saved on Azure ML storage

In [1]:
!az login --tenant cb044e08-488a-4438-8f87-e34637d156f4

[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "cb044e08-488a-4438-8f87-e34637d156f4",
    "id": "16058245-6766-4356-a5f5-eb2511366302",
    "isDefault": true,
    "managedByTenants": [],
    "name": "Azure subscription 1",
    "state": "Enabled",
    "tenantId": "cb044e08-488a-4438-8f87-e34637d156f4",
    "user": {
      "name": "bab232@comcast.net",
      "type": "user"
    }
  }
]


You have logged in. Now let us find all the subscriptions to which you have access...


In [2]:
from azureml.core import Workspace

ws = Workspace.from_config(_file_name='../config/azureml_config.json')

WARNING - Warning: Falling back to use azure cli login credentials.
If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


In [4]:
import mlflow

In [5]:
mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())

- Set Experiment name in mlflow
    - An experiment can have multiple runs so experiment provides an easy way to capture multiple runs

In [36]:
# This API will set the experiment name in MLFLow. If the experiment does't exist then it will create it
mlflow.set_experiment("Taxi_Fares")

INFO: 'Taxi_Fares' does not exist. Creating a new experiment


#### Core ML experiment code

In [7]:
# ADD THIS TO IMPORT MLFLOW FOR SKLEARN MODULES
#import mlflow.sklearn

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [8]:
df = pd.read_csv("../data/nyc-taxi-sample-data.csv")
df.head()

,vendorID,passengerCount,tripDistance,hour_of_day,day_of_week,day_of_month,month_num,normalizeHolidayName,isPaidTimeOff,snowDepth,precipTime,precipDepth,temperature,totalAmount
0,1,1,9.40,15,2,27,1,None,False,29.058824,24.0,3.0,6.185714,44.30
1,2,5,14.75,13,4,15,1,None,False,0.000000,6.0,0.0,4.571930,44.80
2,2,1,3.35,23,4,8,1,None,False,0.000000,1.0,0.0,4.384091,18.96
3,2,1,3.33,18,2,27,1,None,False,29.058824,24.0,3.0,6.185714,16.30
4,2,1,0.47,17,6,3,1,None,False,0.000000,1.0,0.0,3.846429,5.30


In [9]:
df['normalizeHolidayName'].replace("None",0,inplace=True)
df['normalizeHolidayName'].replace("Martin Luther King, Jr. Day",1,inplace=True)
df['normalizeHolidayName'].replace("New Year's Day",2,inplace=True)
df['normalizeHolidayName'].replace("Washington's Birthday",3,inplace=True)
df['normalizeHolidayName'].replace("Memorial Day",4,inplace=True)

In [10]:
df['isPaidTimeOff'].replace("FALSE",0,inplace=True)
df['isPaidTimeOff'].replace("TRUE",1,inplace=True)

In [11]:
#To do a data split and machine learning
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score

In [12]:
x = df[['hour_of_day','day_of_week', 'day_of_month', 'month_num','normalizeHolidayName','isPaidTimeOff','temperature']]
y = df['totalAmount']
       
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30)

In [13]:
x.head()

,hour_of_day,day_of_week,day_of_month,month_num,normalizeHolidayName,isPaidTimeOff,temperature
0,15,2,27,1,0,False,6.185714
1,13,4,15,1,0,False,4.571930
2,23,4,8,1,0,False,4.384091
3,18,2,27,1,0,False,6.185714
4,17,6,3,1,0,False,3.846429


In [14]:
regression = LinearRegression()
pricemodel = regression.fit(x_train, y_train)

In [15]:
#To check the model training is performed
pricemodel

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [16]:
#To predict y values (y predicted) based on the linear model object
y_pred = pricemodel.predict(x_test)
print(y_pred)

[15.60954913 14.62883566 14.37963462 ... 14.65626441 14.84331717
 15.06680703]


### Tracking Parameters, Metrics, and Artifacts
Side Note To log artifacts, we have to save them somewhere before MLflow can log them. This code accomplishes that by using a temporary file that it then deletes.

In [26]:
import os
import mlflow.sklearn
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [37]:
def log_rf(run_name, params, X_train, X_test, y_train, y_test):
# Start run for MLFLOW
  with mlflow.start_run(run_name=run_name) as run:
    # Create model, train it, and create predictions
    rf = RandomForestRegressor(**params)
    rf.fit(X_train, y_train)
    predictions = rf.predict(X_test)

    # Log model to MLFLOW
    mlflow.sklearn.log_model(rf, "random-forest-model")

    # Log params
    mlflow.log_params(params)

    # Create metrics
    mse = mean_squared_error(y_test, predictions)
    mae = mean_absolute_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)

    # Log metrics to MLFLOW
    mlflow.log_metrics({"mse": mse, "mae": mae, "r2": r2})   
    
    print("RandomForestRegressor: mse: ", mse, " mae: ", mae, " r2: ", r2)
    print("RandomForestRegressor RunID: ", run.info.run_uuid)
    
    return run.info.run_uuid

- Run experiment with few tuning parameters now

In [38]:
params = {
  "n_estimators": 102,
  "max_depth": 5,
  "random_state": 42
}

runid_log_rf = log_rf("RandomForestRegressor", params, x_train, x_test, y_train, y_test)

2020/08/24 12:12:59 WARNING mlflow.models.model: Logging model metadata to the tracking server has failed, possibly due older server version. The model artifacts have been logged successfully under azureml://experiments/Taxi_Fares/runs/36106253-1e3d-4fe1-b11c-4620c719caa6/artifacts. In addition to exporting model artifacts, MLflow clients 1.7.0 and above attempt to record model metadata to the  tracking store. If logging to a mlflow server via REST, consider  upgrading the server version to MLflow 1.7.0 or above.


RandomForestRegressor: mse:  113.69246056545698  mae:  7.388034002667485  r2:  0.0029222738761424116
RandomForestRegressor RunID:  36106253-1e3d-4fe1-b11c-4620c719caa6


In [29]:
def log_lr(run_name, X_train, X_test, y_train, y_test):
# Start run for MLFLOW
  with mlflow.start_run(run_name=run_name) as run:
    # Create model, train it, and create predictions
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    predictions = lr.predict(X_test)

    # Log model to MLFLOW
    mlflow.sklearn.log_model(lr, "random-forest-model")

    # Log params
    mlflow.log_params(params)

    # Create metrics
    mse = mean_squared_error(y_test, predictions)
    mae = mean_absolute_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)

    # Log metrics to MLFLOW
    mlflow.log_metrics({"mse": mse, "mae": mae, "r2": r2})   
    
    print("LinearRegression: mse: ", mse, " mae: ", mae, " r2: ", r2)
    print("LinearRegression RunID: ", run.info.run_uuid)
    
    return run.info.run_uuid

In [39]:
runid_log_lr = log_lr("LinearRegression", x_train, x_test, y_train, y_test)

2020/08/24 12:13:07 WARNING mlflow.models.model: Logging model metadata to the tracking server has failed, possibly due older server version. The model artifacts have been logged successfully under azureml://experiments/Taxi_Fares/runs/80222625-9b5c-4aba-978b-b44fbd1f951f/artifacts. In addition to exporting model artifacts, MLflow clients 1.7.0 and above attempt to record model metadata to the  tracking store. If logging to a mlflow server via REST, consider  upgrading the server version to MLflow 1.7.0 or above.


LinearRegression: mse:  113.95118067130882  mae:  7.386593335769577  r2:  0.0006533102741386188
LinearRegression RunID:  80222625-9b5c-4aba-978b-b44fbd1f951f


In [44]:
# Use the same experiment name that was specified above
experiment_name = "Taxi_Fares"

# gets the list of runs for your experiment as an array
exp = ws.experiments[experiment_name]
runs = list(exp.get_runs())
#ws.experiments
exp

Name,Workspace,Report Page,Docs Page
Taxi_Fares,Project,Link to Azure Machine Learning studio,Link to Documentation


In [45]:
runs

[Run(Experiment: Taxi_Fares,
 Id: 80222625-9b5c-4aba-978b-b44fbd1f951f,
 Type: None,
 Status: Completed), Run(Experiment: Taxi_Fares,
 Id: 36106253-1e3d-4fe1-b11c-4620c719caa6,
 Type: None,
 Status: Completed)]

#### Download model artifacts

In [52]:
# Grab this from your training experiment where you logged the model
runid = "80222625-9b5c-4aba-978b-b44fbd1f951f"
model_path = 'random-forest-model'

model_uri='runs:/{}/{}'.format(runid, model_path)

In [53]:
from mlflow.tracking.artifact_utils import _download_artifact_from_uri

absolute_model_path = _download_artifact_from_uri(model_uri, "../deployments/taxiFare_container/")

In [5]:
!az login

[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "cb044e08-488a-4438-8f87-e34637d156f4",
    "id": "16058245-6766-4356-a5f5-eb2511366302",
    "isDefault": true,
    "managedByTenants": [],
    "name": "Azure subscription 1",
    "state": "Enabled",
    "tenantId": "cb044e08-488a-4438-8f87-e34637d156f4",
    "user": {
      "name": "bab232@comcast.net",
      "type": "user"
    }
  }
]


You have logged in. Now let us find all the subscriptions to which you have access...


In [6]:
!az acr login --name FinalProjectCIS395

Login Succeeded


Uppercase characters are detected in the registry name. When using its server url in docker commands, to avoid authentication errors, use all lowercase.


In [4]:
!az acr build --image taxifare --registry FinalProjectCIS395 --file ../deployments/taxifare_container/Dockerfile ../deployments/taxifare_container/

2020/08/24 20:14:48 Downloading source code...

2020/08/24 20:14:49 Finished downloading source code

2020/08/24 20:14:50 Using acb_vol_3b3da815-cf8c-47f3-85e4-e902996a35b0 as the home volume
2020/08/24 20:14:50 Setting up Docker configuration...
2020/08/24 20:14:51 Successfully set up Docker configuration
2020/08/24 20:14:51 Logging in to registry: finalprojectcis395.azurecr.io
2020/08/24 20:14:52 Successfully logged into finalprojectcis395.azurecr.io
2020/08/24 20:14:52 Executing step ID: build. Timeout(sec): 28800, Working directory: '', Network: ''
2020/08/24 20:14:52 Scanning for dependencies...

2020/08/24 20:14:52 Successfully scanned dependencies
2020/08/24 20:14:52 Launching container with name: build
Sending build context to Docker daemon  20.48kB


Step 1/11 : FROM python:3.7.1
3.7.1: Pulling from library/python
54f7e8ac135a: Pulling fs layer

d6341e30912f: Pulling fs layer
087a57faf949: Pulling fs layer
5d71636fb824: Pulling fs layer
0c1db9598990: Pulling fs layer
bfb904e99

Packing source code into tar to upload...
Uploading archived source code from 'C:\Users\bab232\AppData\Local\Temp\build_archive_ede1d86305e64d25b9150b1d0b40f963.tar.gz'...
Sending context (3.367 KiB) to registry: FinalProjectCIS395...
Queued a build with ID: cj3
Waiting for an agent...


In [11]:
!az acr repository list --name FinalProjectCIS395

[
  "cahouseprice",
  "taxifare",
  "taxifare-api"
]


In [8]:
!az aks get-credentials --resource-group Machine_Learning --name taxifafecis395

Merged "taxifafecis395" as current context in C:\Users\bab232\.kube\config


In [9]:
!kubectl get nodes

NAME                                STATUS   ROLES   AGE   VERSION
aks-agentpool-13832284-vmss000000   Ready    agent   35m   v1.16.13
aks-agentpool-13832284-vmss000001   Ready    agent   35m   v1.16.13


In [8]:
!az acr list --resource-group Machine_Learning --query "[].{acrLoginServer:loginServer}" --output table

AcrLoginServer
-----------------------------
finalprojectcis395.azurecr.io


In [13]:
!kubectl apply -f ../deployments/azurekubernetes_deploy/taxifare_aks.yaml

deployment.apps/taxifare-prediction configured
service/taxifare-prediction unchanged


In [10]:
!kubectl get service taxifare-prediction

NAME                  TYPE           CLUSTER-IP    EXTERNAL-IP      PORT(S)          AGE
taxifare-prediction   LoadBalancer   10.0.62.231   52.143.254.242   5010:31923/TCP   18m


In [12]:
!kubectl describe taxifare-prediction

error: the server doesn't have a resource type "taxifare-prediction"
